Before running the next cell please add a shortcut to the shared folder at the root of your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Learning_with_Density_Matrices_and_Random_Features-Supplementary_Material/qmc/tf')
import layers as layers
import models as models

/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.CrossProduct.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrix2Dist.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrixRegression.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
import warnings
import numpy as np
from kdcrf import KDClassifierRF
from kdcrf import RBFSamplerORF
from kdcrf import RBFSamplerSORF
from sklearn.kernel_approximation import RBFSampler

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import gzip

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

In [ ]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer

### example

In [ ]:
!gdown --id 1q1bC5xP5SDYkggj6M5tclu3P0yI3YiTW

Downloading...
From: https://drive.google.com/uc?id=1q1bC5xP5SDYkggj6M5tclu3P0yI3YiTW
To: /content/load_cifar_10.py
100% 3.88k/3.88k [00:00<00:00, 3.65MB/s]


In [ ]:

from load_cifar_10 import cifar10

train_images, train_labels, test_images, test_labels = cifar10(path='data', is_one_hot=False)

print(train_images.shape)
print(np.unique(train_labels))

#%%

train_images.shape

#%%

random_train = np.random.choice(range(train_images.shape[0]), 50000, replace=False)
random_test = np.random.choice(range(test_images.shape[0]), 10000, replace=False)


X_train = train_images[random_train,:]
y_train = train_labels[random_train]
X_test = test_images[random_test,:]
y_test = test_labels[random_test]
#%%


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

Downloaded cifar-10-binary.tar.gz to data
(50000, 3072)
[0 1 2 3 4 5 6 7 8 9]
shape X_train :  (50000, 3072)
shape y_train :  (50000,)
shape X_test :  (10000, 3072)
shape y_test :  (10000,)


In [ ]:
# Transforming each feature into a 0 to 1 range
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=192)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
gammas = [2**i for i in range(-10,10)]

In [ ]:
# Sanity check for shape of data
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)
print("y_train unique: ", np.unique(y_train))
print("y_test unique: ", np.unique(y_test))

X_train:  (50000, 192)
y_train:  (50000,)
X_test:  (10000, 192)
y_test:  (10000,)
y_train unique:  [0 1 2 3 4 5 6 7 8 9]
y_test unique:  [0 1 2 3 4 5 6 7 8 9]


In [ ]:
from sklearn.kernel_approximation import RBFSampler
rbf_feature = RBFSampler(gamma=8, n_components = 1000)
X_train_features = rbf_feature.fit_transform(X_train)
X_test_features = rbf_feature.transform(X_test)

from sklearn.svm import LinearSVC

model_lsvc = LinearSVC(C=2**5,tol=1e-05)
model_lsvc.fit(X_train_features, y_train)
print(model_lsvc.score(X_test_features, y_test))

In [ ]:
input_dim = X_train.shape[1]
num_classes = np.unique(y_train).shape[0]
gamma = 2**-5
component_dim = 1000
random_state=0
num_eig=0
batch_size=8
epochs = 10

In [ ]:
# Create models from qmkdc classifier

def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.QMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=20)

    qmkdc1 = models.QMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, random_state=random_state, fm_x=fm_x)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [ ]:
# Tensorboard 
%load_ext tensorboard
%tensorboard --logdir "/gdrive/My Drive/logs/"

In [ ]:
# Creation of the model without random search. It can be used to verify the initial performance of the model.
import datetime, os
logdir = os.path.join("/gdrive/My Drive/logs", "letters-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_mnist = create_model(input_dim, num_classes, component_dim=1000, gamma=8, lr=1e-04, decay=0.,
                  random_state=None, eig_percentage=1, initialize_with_rff=True,
                  type_of_rff="rff", fix_rff=False)

y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes))
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
model_mnist.fit(X_train, y_train_bin.numpy(), epochs=50, batch_size=batch_size, validation_data=(X_test, y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

In [ ]:
# Final validation test
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
out = model_mnist.predict(X_test)
accuracy_score(y_test, out.argmax(axis=1))

0.72

In [ ]:
# Creation of KerasClassifier for sending to random search
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=16, verbose=1)

In [ ]:
# component dimension
from scipy.stats import randint
#components_dimensions = randint(20,1500)
components_dimensions = [1000]

# gamma
#gammas = [2 ** i for i in range(-25, 25)]
gammas = [2 ** i for i in range(4, 6)]

# number of eigen values
eig_values = [1 / 10, 25 / 100, 50 / 100, 1]

# initialize with rff
#initialize_with_rff = [True, False]
initialize_with_rff = [True]

# type_of_rff_components
type_of_rff = ['rff', 'orf']

# fix_rff
fix_rff = [True]

# learning algorithm parameters
lr = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
decay = [1e-6, 1e-9, 0]

# dictionary summary
param_grid = dict(input_dim=[input_dim], num_classes=[num_classes], component_dim=components_dimensions, gamma=gammas,
                  lr=lr, decay=decay,
                  random_state=[None], eig_percentage=eig_values, initialize_with_rff=initialize_with_rff,
                  type_of_rff=type_of_rff, fix_rff=fix_rff
                  )

In [ ]:
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid,
                              verbose=20, n_iter=1, n_jobs=1)

In [ ]:
# Si no tenemos problema con colab utilizar el siguiente comando
# grid_result = grid.fit(X_train, y_train)
y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes)).numpy()

In [ ]:
cv_results_df = None
n_experiments = 25

for i in range(n_experiments):
  grid_result = grid.fit(X_train, y_train_bin)

  if cv_results_df is None:
    cv_results_df = pd.DataFrame(grid_result.cv_results_)
  else:
    cv_results_df = pd.concat([cv_results_df, pd.DataFrame(grid_result.cv_results_)])

  cv_results_df.to_csv('qmkdc random search.csv')

In [ ]:
experiments = pd.read_csv("cifar.csv")

In [ ]:
experiments.groupby(["experiment", "gamma", "initialize_rhos", "lr", "eig_percentage", "type_of_rff", "fix_rff", "sgd"]).agg({"mean","std", "count"}).reset_index()